# Account - AVR Guest Payments PROD

## Database Summary

In [8]:
Import-Module AWS.Tools.Common
Import-Module AWS.Tools.DynamoDBv2
Import-Module AWS.Tools.RDS

$Profiles   = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match 'DevOps-AVR-Guest-Payments-PROD' }
$Regions    = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"

function Get-AWSDBInformation {
    [CmdletBinding()]
    param (

        $Profiles,
        $Regions,
        $RegexGroup             = 'AccountNumber',
        $RegexGroupAccountName  = 'AccountName',
        $RegexAccountNumber     = "(?<$RegexGroup>\d+):db.+$",
        $RegexAccountNumberDDb  = "(?<$RegexGroup>\d+):table.+$", 
        $RegexAccountName       = "(?<PreFix>DevOps-)(?<$RegexGroupAccountName>.+)"
    )
    
    begin {

        $Summary    = @()
        
    }
    
    process {

        $Summary    = @()
        foreach ($Profile in $Profiles) {

            foreach ($Region in $Regions) {
                
                $RDSInstances   = Get-RDSDBInstance -ProfileName $Profile.ProfileName -Region $Region
                $ddbTables      = Get-DDBTableList -ProfileName $Profile.ProfileName -Region $Region

                switch ([string]::IsNullOrEmpty($RDSInstances)) {
                    $true { 
                        if ($ddbTables) {
                            
                            $Table = $ddbTables | Select-Object -First 1
                            $ddbTable = Get-DDBTable -TableName $Table -ProfileName $Profile.ProfileName -Region $Region
                            $AccountID  = (($ddbTable.TableArn | Select-String -Pattern $RegexAccountNumberDDb).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                        }else {

                            $AccountID = 0
                        }
                    }
                    $false {

                        $RDSInstance   = $RDSInstances | Select-Object -First 1
                        $AccountID  = (($RDSInstance.DBInstanceArn | Select-String -Pattern $RegexAccountNumber).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                    }
                    default {

                        $AccountID = 0
                    }
                }

                $Sum = [PSCustomObject]@{

                    Account             = (($Profile.ProfileName | select-string -Pattern $RegexAccountName).Matches.Groups | Where-Object { $_.Name -eq $RegexGroupAccountName }).Value
                    AccountID           = $AccountID
                    Region              = $Region
                    RDSInstances        = ($RDSInstances.DBInstanceIdentifier | Measure-Object).Count
                    DynamoDBTables      = $ddbTables.Count
                    
                }

                $condition = $Sum.RDSInstances + $Sum.DynamoDBTables
                if ($condition -gt 0) {
                    $Summary += $Sum
                }
            }

        }
        
    }
    
    end {

        $Summary
        
    }
}

$Summary    = Get-AWSDBInformation $Profiles $Regions
$Summary    | Format-Table -AutoSize


Account                 AccountID    Region       RDSInstances DynamoDBTables
-------                 ---------    ------       ------------ --------------
AVR-Guest-Payments-PROD 461191266857 eu-central-1            0              1
AVR-Guest-Payments-PROD 461191266857 eu-west-2               0              5



### DynamoDB Tables

In [9]:
$ddbTables  = @()
$ddbTableList   = @()
$Regions | ForEach-Object   {

    $ddbTableList   = Get-DDBTableList -ProfileName $Profiles.ProfileName -Region $_
    $ddbBackupList  = Get-DDBBackupList -ProfileName $Profiles.ProfileName -Region $_

    foreach ($ddbTable in $ddbTableList) {
        
        $Table = Get-DDBTable -TableName $ddbTable -ProfileName $Profiles.ProfileName -Region $_

        if ($ddbBackupList) {
            
            $BackupAvailable = $ddbBackupList.TableName.Contains("$($Table.TableName)")
        }
        else {

            $BackupAvailable = $false
        }
        

        $Sum = [PSCustomObject]@{

            Region              = $_
            TableName           = $Table.TableName
            Backup              = $BackupAvailable
            CreationDateTime    = $Table.CreationDateTime
            ItemCount           = $Table.ItemCount
            # TableSizeBytes      = $Table.TableSizeBytes
            SizeMB         = "$([math]::round($($Table.TableSizeBytes /1MB), 0)) MB"
            Status         = $Table.TableStatus
            # TableId             = $Table.TableId
            # KeySchema           = $Table.KeySchema
            # GlobalSecondaryIndexes = $Table.GlobalSecondaryIndexes
            
        }
        $ddbTables += $Sum
    }
    
    

}
$ddbTables | Format-Table -AutoSize


Region       TableName                            Backup CreationDateTime      ItemCount SizeMB  St
                                                                                                 at
                                                                                                 us
------       ---------                            ------ ----------------      --------- ------  --
eu-central-1 transaction                           False 1/29/2021 6:01:02 PM          8 0 MB    A…
eu-west-2    instalment-plans-payment-instalments  False 6/9/2023 11:17:17 AM     244029 76 MB   A…
eu-west-2    scheduled-payments                    False 5/24/2022 11:59:53 AM      2881 0 MB    A…
eu-west-2    terraform-locks                       False 10/7/2021 8:08:07 AM         27 0 MB    A…
eu-west-2    transaction                           False 4/28/2021 11:30:47 AM   5394241 4735 MB A…
eu-west-2    vt-payment-links                      False 7/12/2022 12:51:42 PM    505932 63 MB   A…